In [ ]:
import os
import numpy as np

import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cd /content/drive/MyDrive/MiningProject
os.chdir('/content/drive/MyDrive/MiningProject')

In [ ]:
day_data = '/content/drive/MyDrive/MiningProject/day_data.csv'

complete_df = pd.read_csv(day_data, header=0)
print(complete_df)

        dateTime  laminutes  maminutes  ...  steps_std  steps_sum  pid
0     2019-11-02      202.0         43  ...  22.935518      13118    1
1     2019-11-03      262.0         63  ...  22.301633      14312    1
2     2019-11-04      230.0         23  ...  21.149870      10970    1
3     2019-11-05      263.0         22  ...  29.787397      16186    1
4     2019-11-06      206.0         10  ...  19.224741       8189    1
...          ...        ...        ...  ...        ...        ...  ...
1666  2020-03-26       49.0          0  ...   5.485446        700   16
1667  2020-03-27       93.0          4  ...  13.252194       2070   16
1668  2020-03-28       52.0          0  ...   4.030354        589   16
1669  2020-03-29       90.0          0  ...   8.909012       1714   16
1670  2020-03-30       69.0          9  ...  13.344122       1773   16

[1671 rows x 50 columns]


In [ ]:
print(complete_df.columns)

Index(['dateTime', 'laminutes', 'maminutes', 'vaminutes', 'sedminutes',
       'resthr', 'logId', 'dateOfSleep', 'startTime', 'endTime', 'duration',
       'minutesToFallAsleep', 'minutesAsleep', 'minutesAwake',
       'minutesAfterWakeup', 'timeInBed', 'timestamp', 'sleep_log_entry_id',
       'overall_score', 'composition_score', 'revitalization_score',
       'duration_score', 'deep_sleep_in_minutes', 'resting_heart_rate',
       'restlessness', 'hr_max', 'hr_min', 'hr_mean', 'hr_median', 'hr_std',
       'hr_sum', 'cal_max', 'cal_min', 'cal_mean', 'cal_median', 'cal_std',
       'cal_sum', 'dist_max', 'dist_min', 'dist_mean', 'dist_median',
       'dist_std', 'dist_sum', 'steps_max', 'steps_min', 'steps_mean',
       'steps_median', 'steps_std', 'steps_sum', 'pid'],
      dtype='object')


In [ ]:
dict_of_df = {}

for i in range(0,15):
  for x in complete_df.pid.unique(): 
    pid_df = complete_df[complete_df['pid']==x]
    dict_of_df["df_{}".format(x)] = pid_df
    
print(dict_of_df['df_1'])

       dateTime  laminutes  maminutes  ...  steps_std  steps_sum  pid
0    2019-11-02      202.0         43  ...  22.935518      13118    1
1    2019-11-03      262.0         63  ...  22.301633      14312    1
2    2019-11-04      230.0         23  ...  21.149870      10970    1
3    2019-11-05      263.0         22  ...  29.787397      16186    1
4    2019-11-06      206.0         10  ...  19.224741       8189    1
..          ...        ...        ...  ...        ...        ...  ...
149  2020-03-26      142.0         26  ...  17.299613       7357    1
150  2020-03-27      181.0         28  ...  22.421354      11532    1
151  2020-03-28      207.0         69  ...  26.484124      16448    1
152  2020-03-29      145.0         19  ...  16.060223       6698    1
153  2020-03-30      161.0          6  ...  12.070003       4761    1

[154 rows x 50 columns]


# **LOOP OF ALL OPERATIONS** 


Creates individual dataframes of necessary information within the discretized bins. 


*   Needs PID and dateTime columns added back in
*   Doesnt have the dateTime set as index and type changed 



In [ ]:
from scipy import stats
from scipy.stats import zscore

dict_bin = {}

for i in range(1,17):
  dict_of_df['df_{}'.format(i)].dateTime = pd.to_datetime(dict_of_df['df_{}'.format(i)].dateTime)
  dict_of_df['df_{}'.format(i)].set_index('dateTime', inplace = True)
  med = dict_of_df['df_{}'.format(i)].rolling(2, min_periods=1).median()
  med = med.drop(columns=['minutesToFallAsleep','dist_min','dist_median','steps_min','steps_median','pid'])
  z = med.apply(zscore)
  #print(z) --- Still a dataframe
  names = list(z.columns)
  for n in names:
    z[n] = np.digitize(z[n], [-0.333, 0.333])
  z = z.assign(pid=i)
  dict_bin["z_{}".format(i)] = z
  #display(dict_bin["z_{}".format(i)])


comp = pd.concat([dict_bin["z_1"], dict_bin["z_2"], dict_bin["z_3"], dict_bin["z_4"], dict_bin["z_5"], dict_bin["z_6"], dict_bin["z_7"], dict_bin["z_8"], dict_bin["z_9"], dict_bin["z_10"], dict_bin["z_11"], dict_bin["z_13"],dict_bin["z_14"],dict_bin["z_15"],dict_bin["z_16"]], axis = 0)#, ignore_index=True)
print(comp)
comp.to_csv("final_bins.csv", index=False)


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:2419: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


            laminutes  maminutes  vaminutes  ...  steps_std  steps_sum  pid
dateTime                                     ...                           
2019-11-02          0          2          1  ...          1          1    1
2019-11-03          1          2          1  ...          1          2    1
2019-11-04          1          2          0  ...          0          1    1
2019-11-05          1          0          1  ...          2          1    1
2019-11-06          1          0          0  ...          2          1    1
...               ...        ...        ...  ...        ...        ...  ...
2020-03-26          0          2          2  ...          0          0   16
2020-03-27          0          1          1  ...          0          0   16
2020-03-28          0          1          1  ...          0          0   16
2020-03-29          0          0          0  ...          0          0   16
2020-03-30          0          1          1  ...          1          1   16

[1670 rows 

In [ ]:
print(dict_bin["z_4"])

            laminutes  maminutes  vaminutes  ...  steps_std  steps_sum  pid
dateTime                                     ...                           
2020-02-26          2          2          1  ...          2          2    4
2020-02-26          2          2          1  ...          2          2    4
2020-02-27          2          1          2  ...          2          2    4
2020-02-28          2          0          1  ...          2          2    4
2020-02-29          2          0          0  ...          1          2    4
2020-03-01          2          2          2  ...          2          2    4
2020-03-01          0          2          2  ...          2          2    4
2020-03-02          0          1          2  ...          0          0    4
2020-03-02          0          0          0  ...          0          0    4
2020-03-02          0          0          0  ...          0          0    4
2020-03-03          0          0          0  ...          0          0    4
2020-03-04  

In [ ]:
#dict_of_df['df_1'].set_index('dateTime')

med_1 = dict_of_df['df_1'].rolling(2, min_periods=1).median()
display(med_1)

,laminutes,maminutes,vaminutes,sedminutes,resthr,logId,duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,timeInBed,sleep_log_entry_id,overall_score,composition_score,revitalization_score,duration_score,deep_sleep_in_minutes,resting_heart_rate,restlessness,hr_max,hr_min,hr_mean,hr_median,hr_std,hr_sum,cal_max,cal_min,cal_mean,cal_median,cal_std,cal_sum,dist_max,dist_min,dist_mean,dist_median,dist_std,dist_sum,steps_max,steps_min,steps_mean,steps_median,steps_std,steps_sum,pid
dateTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-11-02,202.0,43.0,56.0,709.0,52.881497,2.448601e+10,25800000.0,0.0,378.0,52.0,0.0,430.0,2.448601e+10,79.0,20.0,22.0,37.0,36.0,53.0,0.078978,122.0,44.0,62.602226,57.0,15.280065,635663.0,11.400,1.39,2.453861,1.39,2.170251,3533.560,9540.0,0.0,735.055556,0.0,1852.161856,1058480.0,118.0,0.0,9.109722,0.0,22.935518,13118.0,1.0
2019-11-03,232.0,53.0,50.5,678.5,53.051761,2.449168e+10,25560000.0,0.0,378.0,48.0,0.0,426.0,2.449168e+10,75.0,18.0,19.5,37.5,37.0,52.5,0.086235,127.0,44.5,63.577033,58.0,14.906456,642204.5,11.400,1.39,2.528573,1.46,2.156061,3641.145,9540.0,0.0,765.750502,0.0,1816.181277,1102282.5,118.0,0.0,9.527759,0.0,22.618576,13715.0,1.0
2019-11-04,246.0,43.0,36.5,704.0,53.766582,2.450263e+10,24630000.0,0.0,369.5,41.0,0.0,410.5,2.450263e+10,69.5,15.5,16.0,38.0,37.5,52.5,0.079171,129.0,44.5,64.051975,59.0,14.013860,652143.0,11.955,1.39,2.466010,1.53,2.054182,3551.055,9825.0,0.0,706.064976,0.0,1744.944670,1016027.5,121.5,0.0,8.784573,0.0,21.725752,12641.0,1.0
2019-11-05,246.5,22.5,42.0,748.5,53.285126,2.451491e+10,22830000.0,0.0,343.5,37.0,0.5,380.5,2.451491e+10,70.5,15.0,18.5,37.0,31.5,53.5,0.079018,144.5,44.0,65.112319,59.0,18.565347,693535.5,14.940,1.39,2.481948,1.53,2.294556,3574.005,13970.0,0.0,784.047113,0.0,2189.073969,1128720.0,148.0,0.0,9.431814,0.0,25.468633,13578.0,1.0
2019-11-06,234.5,16.0,33.5,793.0,53.231150,2.452797e+10,21780000.0,0.0,333.5,29.5,1.0,363.0,2.452797e+10,70.0,16.0,17.5,36.5,45.5,53.0,0.078828,144.0,45.0,65.279506,60.0,17.159461,710752.5,17.790,1.39,2.426056,1.53,2.298337,3493.520,14720.0,0.0,702.806668,0.0,2105.811361,1011815.0,150.0,0.0,8.465518,0.0,24.506069,12187.5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-26,164.0,18.5,42.5,784.0,50.457735,2.647319e+10,25860000.0,0.0,381.0,50.0,0.0,431.0,2.647319e+10,73.5,16.5,19.5,37.5,60.0,50.0,0.062861,123.0,43.0,60.534716,57.0,13.000874,635888.0,12.555,1.38,2.175194,1.38,1.865232,3132.280,10260.0,0.0,462.417794,0.0,1528.186191,665675.0,118.0,0.0,5.723303,0.0,18.868475,8239.0,1.0
2020-03-27,161.5,27.0,53.5,796.0,50.426959,2.648542e+10,24120000.0,0.0,361.0,41.0,0.0,402.0,2.648542e+10,74.0,17.5,20.0,36.5,56.5,50.0,0.077633,129.0,43.5,60.865225,57.5,13.406499,644359.5,13.105,1.38,2.252743,1.45,1.979302,3243.950,10545.0,0.0,526.803210,0.0,1603.091173,758390.0,121.5,0.0,6.560456,0.0,19.860483,9444.5,1.0
2020-03-28,194.0,48.5,74.0,750.0,50.678841,2.649708e+10,22410000.0,0.0,330.5,43.0,0.0,373.5,2.649708e+10,69.5,17.5,17.0,35.0,32.0,50.0,0.092044,133.5,45.0,63.165751,58.5,14.895503,680779.0,13.660,1.38,2.552618,1.52,2.316556,3675.770,10230.0,0.0,780.756944,0.0,1968.338737,1124290.0,126.5,0.0,9.715278,0.0,24.452739,13990.0,1.0


# **EVERYTHING BELOW IS UNECESSARY**

In [ ]:
med_2 = dict_of_df['df_2'].rolling(2, min_periods=1).median()
med_3 = dict_of_df['df_3'].rolling(2, min_periods=1).median()
med_4 = dict_of_df['df_4'].rolling(2, min_periods=1).median()
med_5 = dict_of_df['df_5'].rolling(2, min_periods=1).median()
med_6 = dict_of_df['df_6'].rolling(2, min_periods=1).median()
med_7 = dict_of_df['df_7'].rolling(2, min_periods=1).median()
med_8 = dict_of_df['df_8'].rolling(2, min_periods=1).median()
med_9 = dict_of_df['df_9'].rolling(2, min_periods=1).median()
med_10 = dict_of_df['df_10'].rolling(2, min_periods=1).median()
med_11 = dict_of_df['df_11'].rolling(2, min_periods=1).median()
med_12 = dict_of_df['df_12'].rolling(2, min_periods=1).median()
#df_13 = dict_of_df['df_13'].drop(columns = ['resthr'])
med_13 = dict_of_df['df_13'].rolling(2, min_periods=1).median()
med_14 = dict_of_df['df_14'].rolling(2, min_periods=1).median()
med_15 = dict_of_df['df_15'].rolling(2, min_periods=1).median()
med_16 = dict_of_df['df_16'].rolling(2, min_periods=1).median()

#display(med_16)

In [ ]:
from scipy import stats
from scipy.stats import zscore

In [ ]:
p1_z = med_1.drop(columns=['minutesToFallAsleep','dist_min','dist_median','steps_min','steps_median','pid'])
z_score_1 = stats.zscore(p1_z, nan_policy='omit')
Z_score_test = p1_z.apply(zscore)

In [ ]:
np.set_printoptions(edgeitems=10, linewidth=1000, formatter=dict(float=lambda x: "%.3g" % x))
#display(z_score_1)
#p1_z.describe()

In [ ]:
display(Z_score_test)

,laminutes,maminutes,vaminutes,sedminutes,resthr,logId,duration,minutesAsleep,minutesAwake,minutesAfterWakeup,timeInBed,sleep_log_entry_id,overall_score,composition_score,revitalization_score,duration_score,deep_sleep_in_minutes,resting_heart_rate,restlessness,hr_max,hr_min,hr_mean,hr_median,hr_std,hr_sum,cal_max,cal_min,cal_mean,cal_median,cal_std,cal_sum,dist_max,dist_mean,dist_std,dist_sum,steps_max,steps_mean,steps_std,steps_sum
dateTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-11-02,-0.688294,0.974899,0.313650,-0.078634,0.496062,-1.613698,0.764340,0.634437,1.181067,-0.671137,0.764340,-1.613698,1.800915,1.649482,2.081377,0.147905,-0.250849,0.970408,-0.359723,-1.250864,0.133035,-0.554443,-0.945902,-0.136894,-0.600454,-1.558355,0.487859,-0.215389,-1.164470,-0.164162,-0.215389,-0.925343,0.098215,-0.128515,0.099304,-0.980256,0.143066,0.001859,0.144202
2019-11-03,-0.249896,1.628368,0.074100,-0.435661,0.637734,-1.604338,0.666687,0.634437,0.622926,-0.671137,0.666687,-1.604338,0.827362,0.455497,0.867240,0.358807,-0.172490,0.559851,-0.037865,-0.883244,0.324505,-0.252133,-0.577120,-0.244543,-0.486616,-1.558355,0.487859,0.068957,-0.374726,-0.203313,0.068957,-0.925343,0.268704,-0.234070,0.268373,-0.980256,0.335837,-0.086192,0.335512
2019-11-04,-0.045311,0.974899,-0.535666,-0.137163,1.232521,-1.586243,0.288282,0.402330,-0.353822,-0.671137,0.288282,-1.586243,-0.511273,-1.036983,-0.832551,0.569709,-0.133311,0.559851,-0.351160,-0.736196,0.324505,-0.104843,-0.208338,-0.501728,-0.313662,-1.318662,0.487859,-0.169150,0.415019,-0.484399,-0.169150,-0.792293,-0.062808,-0.443053,-0.064555,-0.735079,-0.006871,-0.334230,-0.008653
2019-11-05,-0.038004,-0.364712,-0.296115,0.383745,0.831912,-1.565955,-0.444114,-0.307643,-0.911963,-0.384835,-0.444114,-1.565955,-0.267885,-1.335479,0.381586,0.147905,-0.603462,1.380965,-0.357950,0.403428,0.133035,0.223993,-0.208338,0.809700,0.406667,-0.029502,0.487859,-0.108493,0.415019,0.178798,-0.108493,1.142764,0.370329,0.859870,0.370417,1.121268,0.291593,0.705593,0.291610
2019-11-06,-0.213363,-0.789467,-0.666331,0.904654,0.786999,-1.544372,-0.871346,-0.580710,-1.958479,-0.098533,-0.871346,-1.544372,-0.389579,-0.738487,-0.104069,-0.062997,0.493556,0.970408,-0.366354,0.366666,0.515976,0.275842,0.160444,0.404619,0.706285,1.201354,0.487859,-0.321214,0.415019,0.189231,-0.321214,1.492895,-0.080905,0.615606,-0.080814,1.261370,-0.153997,0.438179,-0.153978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-26,-1.243598,-0.626100,-0.274338,0.799301,-1.520697,1.670043,0.788753,0.716357,0.901997,-0.671137,0.788753,1.670043,0.462280,-0.439991,0.867240,0.358807,1.629753,-1.492935,-1.074580,-1.177340,-0.249906,-1.195624,-0.945902,-0.793602,-0.596539,-1.059535,0.415653,-1.275966,-1.277291,-1.005717,-1.275966,-0.589217,-1.416100,-1.078949,-1.416848,-0.980256,-1.418523,-1.128020,-1.419283
2020-03-27,-1.280131,-0.070651,0.204764,0.939771,-1.546305,1.690251,0.080770,0.170223,-0.353822,-0.671137,0.080770,1.690251,0.583974,0.157001,1.110068,-0.062997,1.355499,-1.492935,-0.419359,-0.736196,-0.058436,-1.093126,-0.761511,-0.676728,-0.449114,-0.822001,0.415653,-0.980824,-0.487546,-0.690996,-0.980824,-0.456168,-1.058484,-0.859203,-1.058986,-0.735079,-1.032484,-0.852427,-1.032978
2020-03-28,-0.805200,1.334307,1.097635,0.401304,-1.336720,1.709513,-0.615007,-0.662630,-0.074751,-0.671137,-0.615007,1.709513,-0.511273,0.157001,-0.346896,-0.695702,-0.564282,-1.492935,0.219787,-0.405337,0.515976,-0.379681,-0.392729,-0.247699,0.184673,-0.582308,0.415653,0.160470,0.302198,0.239496,0.160470,-0.603223,0.352055,0.212308,0.353318,-0.384825,0.422308,0.423364,0.423636


In [ ]:
zbin1 = np.digitize(Z_score_test, [-0.333, 0.333])
print(zbin1)
#for i in zbin1: 
 # unique, counts = np.unique(zbin1[i], return_counts=True)
  #print(np.asarray((unique, counts)).T)

[[0 2 1 1 2 0 2 2 2 0 ... 1 1 0 1 1 1 0 1 1 1]
 [1 2 1 0 2 0 2 2 2 0 ... 1 1 0 1 1 1 0 2 1 2]
 [1 2 0 1 2 0 1 2 0 0 ... 0 1 0 1 0 1 0 1 0 1]
 [1 0 1 2 2 0 0 1 0 0 ... 1 1 2 2 2 2 2 1 2 1]
 [1 0 0 2 2 0 0 0 0 1 ... 1 1 2 1 2 1 2 1 2 1]
 [0 0 0 2 2 0 0 0 0 0 ... 0 0 2 0 0 0 1 0 0 0]
 [0 0 0 2 2 0 0 0 2 0 ... 0 0 2 0 0 0 1 0 0 0]
 [0 1 0 2 2 0 0 0 1 0 ... 0 0 1 0 0 0 1 0 1 0]
 [1 2 2 1 2 0 1 1 0 0 ... 1 2 0 2 1 2 0 2 2 2]
 [2 1 2 0 2 0 1 1 1 0 ... 2 2 0 2 2 2 0 2 2 2]
 ...
 [0 1 1 2 0 2 1 1 1 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 2 2 2 2 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 2 0 2 2 2 2 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 2 0 2 2 2 2 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 2 0 2 2 2 2 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 2 0 2 2 2 2 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 2 0 2 1 1 0 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 2 2 2 0 2 0 0 1 0 ... 1 1 0 2 1 2 0 2 2 2]
 [0 2 1 2 1 2 2 1 2 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 2 1 2 2 2 2 1 ... 0 0 0 0 0 0 0 0 0 0]]


In [ ]:
names = list(Z_score_test.columns)
print(names)

['laminutes', 'maminutes', 'vaminutes', 'sedminutes', 'resthr', 'logId', 'duration', 'minutesAsleep', 'minutesAwake', 'minutesAfterWakeup', 'timeInBed', 'sleep_log_entry_id', 'overall_score', 'composition_score', 'revitalization_score', 'duration_score', 'deep_sleep_in_minutes', 'resting_heart_rate', 'restlessness', 'hr_max', 'hr_min', 'hr_mean', 'hr_median', 'hr_std', 'hr_sum', 'cal_max', 'cal_min', 'cal_mean', 'cal_median', 'cal_std', 'cal_sum', 'dist_max', 'dist_mean', 'dist_std', 'dist_sum', 'steps_max', 'steps_mean', 'steps_std', 'steps_sum']


In [ ]:
for i in names:
  Z_score_test[i] = np.digitize(Z_score_test[i], [-0.333, 0.333])
  
display(Z_score_test)

,laminutes,maminutes,vaminutes,sedminutes,resthr,logId,duration,minutesAsleep,minutesAwake,minutesAfterWakeup,timeInBed,sleep_log_entry_id,overall_score,composition_score,revitalization_score,duration_score,deep_sleep_in_minutes,resting_heart_rate,restlessness,hr_max,hr_min,hr_mean,hr_median,hr_std,hr_sum,cal_max,cal_min,cal_mean,cal_median,cal_std,cal_sum,dist_max,dist_mean,dist_std,dist_sum,steps_max,steps_mean,steps_std,steps_sum
dateTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-11-02,0,2,1,1,2,0,2,2,2,0,2,0,2,2,2,1,1,2,0,0,1,0,0,1,0,0,2,1,0,1,1,0,1,1,1,0,1,1,1
2019-11-03,1,2,1,0,2,0,2,2,2,0,2,0,2,2,2,2,1,2,1,0,1,1,0,1,0,0,2,1,0,1,1,0,1,1,1,0,2,1,2
2019-11-04,1,2,0,1,2,0,1,2,0,0,1,0,0,0,0,2,1,2,0,0,1,1,1,0,1,0,2,1,2,0,1,0,1,0,1,0,1,0,1
2019-11-05,1,0,1,2,2,0,0,1,0,0,0,0,1,0,2,1,0,2,0,2,1,1,1,2,2,1,2,1,2,1,1,2,2,2,2,2,1,2,1
2019-11-06,1,0,0,2,2,0,0,0,0,1,0,0,0,0,1,1,2,2,0,2,2,1,1,2,2,2,2,1,2,1,1,2,1,2,1,2,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-26,0,0,1,2,0,2,2,2,2,0,2,2,2,0,2,2,2,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0
2020-03-27,0,1,1,2,0,2,1,1,0,0,1,2,2,1,2,1,2,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0
2020-03-28,0,2,2,2,0,2,0,0,1,0,0,2,0,1,0,0,0,0,1,0,2,0,0,1,1,0,2,1,1,1,1,0,2,1,2,0,2,2,2


In [ ]:
#med_1.describe()

In [ ]:
display(z_score_1)

array([[-0.688, 0.975, 0.314, -0.0786, 0.496, -1.61, 0.764, 0.634, 1.18, -0.671, ..., -0.164, -0.215, -0.925, 0.0982, -0.129, 0.0993, -0.98, 0.143, 0.00186, 0.144],
       [-0.25, 1.63, 0.0741, -0.436, 0.638, -1.6, 0.667, 0.634, 0.623, -0.671, ..., -0.203, 0.069, -0.925, 0.269, -0.234, 0.268, -0.98, 0.336, -0.0862, 0.336],
       [-0.0453, 0.975, -0.536, -0.137, 1.23, -1.59, 0.288, 0.402, -0.354, -0.671, ..., -0.484, -0.169, -0.792, -0.0628, -0.443, -0.0646, -0.735, -0.00687, -0.334, -0.00865],
       [-0.038, -0.365, -0.296, 0.384, 0.832, -1.57, -0.444, -0.308, -0.912, -0.385, ..., 0.179, -0.108, 1.14, 0.37, 0.86, 0.37, 1.12, 0.292, 0.706, 0.292],
       [-0.213, -0.789, -0.666, 0.905, 0.787, -1.54, -0.871, -0.581, -1.96, -0.0985, ..., 0.189, -0.321, 1.49, -0.0809, 0.616, -0.0808, 1.26, -0.154, 0.438, -0.154],
       [-0.805, -1.15, -1.56, 1.66, 1.13, -1.53, -0.823, -0.772, -0.703, -0.385, ..., -0.932, -1.18, 0.447, -1.37, -0.882, -1.37, -0.0696, -1.35, -0.891, -1.35],
       [-0.981,

In [ ]:
display(med_2)
p2_z = med_2.drop(columns=['minutesToFallAsleep','minutesAfterWakeup','dist_min','dist_median','steps_min','steps_median','pid'])
z_score_2 = stats.zscore(p2_z, nan_policy='omit')

,laminutes,maminutes,vaminutes,sedminutes,resthr,logId,duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,timeInBed,sleep_log_entry_id,overall_score,composition_score,revitalization_score,duration_score,deep_sleep_in_minutes,resting_heart_rate,restlessness,hr_max,hr_min,hr_mean,hr_median,hr_std,hr_sum,cal_max,cal_min,cal_mean,cal_median,cal_std,cal_sum,dist_max,dist_min,dist_mean,dist_median,dist_std,dist_sum,steps_max,steps_min,steps_mean,steps_median,steps_std,steps_sum,pid
dateTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-02,240.0,16.0,48.0,628.0,45.674418,2.526647e+10,30480000.0,0.0,425.0,83.0,0.0,508.0,2.526647e+10,72.0,21.0,13.0,38.0,56.0,45.0,0.169125,161.0,38.0,62.588071,54.0,24.953748,627508.0,15.650,1.2,2.080507,1.32,2.009934,2995.930,7470.0,0.0,806.990679,0.0,1437.815936,606050.0,100.0,0.0,11.203728,0.0,19.759279,8414.0,2.0
2020-01-03,260.5,14.0,55.5,677.0,46.770770,2.526647e+10,25980000.0,0.0,372.5,60.5,0.0,433.0,2.526647e+10,67.0,20.5,10.5,36.0,40.5,45.0,0.207993,140.0,39.5,60.269198,54.0,19.063114,558739.0,13.605,1.2,2.169993,1.38,2.012111,3124.790,8065.0,0.0,1071.450148,0.0,1898.306884,879470.0,108.0,0.0,14.605920,0.0,25.753992,11977.5,2.0
2020-01-04,215.5,16.5,98.5,727.0,47.103292,2.526771e+10,12720000.0,0.0,191.0,21.0,0.0,212.0,2.526895e+10,64.0,20.5,12.0,31.5,29.5,46.0,0.243108,131.0,40.0,58.193032,53.0,16.105417,501196.5,12.825,1.2,2.317031,1.38,2.277737,3336.525,10610.0,0.0,1533.776632,0.0,2597.135525,1203300.0,133.5,0.0,20.711929,0.0,34.987991,16247.0,2.0
2020-01-04,150.0,21.0,134.0,728.0,46.339462,2.527019e+10,12210000.0,0.0,176.0,27.5,0.0,203.5,2.527143e+10,66.0,21.0,16.0,29.0,34.0,47.0,0.239354,143.0,39.0,58.435740,52.0,19.038355,512423.0,14.090,1.2,2.374583,1.32,2.541186,3419.400,12560.0,0.0,1731.643646,0.0,2835.473216,1253710.0,151.0,0.0,23.415746,0.0,38.227279,16953.0,2.0
2020-01-05,146.5,26.5,120.0,671.5,45.993640,2.527709e+10,26550000.0,0.0,377.0,65.5,0.0,442.5,2.527709e+10,70.0,20.5,15.5,34.0,52.0,46.5,0.211504,138.0,38.5,57.344854,51.0,18.142878,507835.5,15.110,1.2,2.339014,1.26,2.543989,3368.180,10835.0,0.0,1676.067855,0.0,2734.338371,1137310.0,136.5,0.0,22.568190,0.0,36.749675,15318.5,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-26,117.0,12.5,63.0,815.0,47.557917,2.647481e+10,25950000.0,0.0,370.0,62.5,0.0,432.5,2.647481e+10,72.0,19.5,18.0,34.5,34.0,47.5,0.190135,160.5,39.0,66.079118,54.0,30.190734,668464.0,16.070,1.2,1.985024,1.20,2.439581,2858.435,10655.0,0.0,1085.190599,0.0,2480.206537,594290.0,124.5,0.0,13.687384,0.0,29.600862,7459.5,2.0
2020-03-27,149.5,13.0,81.5,767.5,47.517803,2.648287e+10,25710000.0,0.0,357.5,71.0,0.0,428.5,2.648287e+10,67.5,19.5,16.5,31.5,36.0,47.0,0.209669,163.5,39.5,66.683499,55.5,27.580543,651814.5,16.195,1.2,2.151288,1.20,2.503485,3097.855,10455.0,0.0,1113.119159,0.0,2109.931131,731115.0,122.5,0.0,15.468042,0.0,28.317595,10090.0,2.0
2020-03-28,204.0,27.0,100.0,644.5,48.387552,2.649681e+10,27870000.0,0.0,375.0,89.5,0.0,464.5,2.649681e+10,63.5,19.5,13.5,30.5,69.5,47.0,0.196709,140.0,40.0,62.052220,56.0,19.661699,614189.5,14.030,1.2,2.369215,1.26,2.446962,3411.670,11355.0,0.0,1561.643217,0.0,2721.727284,1199275.0,134.5,0.0,20.974351,0.0,35.638404,16105.5,2.0


/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:2419: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


In [ ]:
p3_z = med_3.drop(columns=['minutesToFallAsleep','dist_min','steps_min','pid'])
z_score_3 = stats.zscore(p3_z, nan_policy='omit')

In [ ]:
p4_z = med_4.drop(columns=['minutesToFallAsleep','dist_min','steps_min','pid'])
z_score_4 = stats.zscore(p4_z, nan_policy='omit')

In [ ]:
p5_z = med_5.drop(columns=['minutesToFallAsleep','dist_min','steps_min','pid'])
z_score_5 = stats.zscore(p5_z, nan_policy='omit')

In [ ]:
p6_z = med_6.drop(columns=['minutesToFallAsleep','dist_min','steps_min','pid'])
z_score_6 = stats.zscore(p6_z, nan_policy='omit')

In [ ]:
p7_z = med_7.drop(columns=['minutesToFallAsleep','dist_min','steps_min','pid'])
z_score_7 = stats.zscore(p7_z, nan_policy='omit')

In [ ]:
p8_z = med_8.drop(columns=['minutesToFallAsleep','dist_min','steps_min','pid'])
z_score_8 = stats.zscore(p8_z, nan_policy='omit')

In [ ]:
p9_z = med_9.drop(columns=['minutesToFallAsleep','dist_min','steps_min','pid'])
z_score_9 = stats.zscore(p9_z, nan_policy='omit')

In [ ]:
p10_z = med_10.drop(columns=['minutesToFallAsleep','dist_min','steps_min','pid'])
z_score_10 = stats.zscore(p10_z, nan_policy='omit')

In [ ]:
p11_z = med_11.drop(columns=['minutesToFallAsleep','dist_min','steps_min','pid'])
z_score_11 = stats.zscore(p11_z, nan_policy='omit')

**NOT GOOD DATA**

The rolling mean creates one row of data, something is wrong with this particular person. 

In [ ]:
med_12.describe()

,laminutes,maminutes,vaminutes,sedminutes,resthr,logId,duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,timeInBed,sleep_log_entry_id,overall_score,composition_score,revitalization_score,duration_score,deep_sleep_in_minutes,resting_heart_rate,restlessness,hr_max,hr_min,hr_mean,hr_median,hr_std,hr_sum,cal_max,cal_min,cal_mean,cal_median,cal_std,cal_sum,dist_max,dist_min,dist_mean,dist_median,dist_std,dist_sum,steps_max,steps_min,steps_mean,steps_median,steps_std,steps_sum,pid
count,0.0,1.0,1.0,1.0,0.0,1.000000e+00,1.0,1.0,1.0,1.0,1.0,1.0,1.000000e+00,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.000000,1.0,1.000000,1.0,1.00,1.00,1.000000,1.00,1.000000,1.00,1.0,1.0,1.000000,1.0,1.000000,1.0,1.0,1.0,1.000000,1.0,1.000000,1.0,1.0
mean,NaN,23.0,5.0,499.0,NaN,2.591802e+10,36060000.0,0.0,535.0,66.0,1.0,601.0,2.591802e+10,85.0,21.0,19.0,45.0,107.0,61.0,0.051089,133.0,50.0,72.757389,70.0,13.400941,706547.0,8.63,1.17,1.984035,1.28,1.384604,2857.01,8440.0,0.0,891.184713,0.0,1557.482515,699580.0,113.0,0.0,11.959236,0.0,20.871592,9388.0,12.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,23.0,5.0,499.0,NaN,2.591802e+10,36060000.0,0.0,535.0,66.0,1.0,601.0,2.591802e+10,85.0,21.0,19.0,45.0,107.0,61.0,0.051089,133.0,50.0,72.757389,70.0,13.400941,706547.0,8.63,1.17,1.984035,1.28,1.384604,2857.01,8440.0,0.0,891.184713,0.0,1557.482515,699580.0,113.0,0.0,11.959236,0.0,20.871592,9388.0,12.0
25%,NaN,23.0,5.0,499.0,NaN,2.591802e+10,36060000.0,0.0,535.0,66.0,1.0,601.0,2.591802e+10,85.0,21.0,19.0,45.0,107.0,61.0,0.051089,133.0,50.0,72.757389,70.0,13.400941,706547.0,8.63,1.17,1.984035,1.28,1.384604,2857.01,8440.0,0.0,891.184713,0.0,1557.482515,699580.0,113.0,0.0,11.959236,0.0,20.871592,9388.0,12.0
50%,NaN,23.0,5.0,499.0,NaN,2.591802e+10,36060000.0,0.0,535.0,66.0,1.0,601.0,2.591802e+10,85.0,21.0,19.0,45.0,107.0,61.0,0.051089,133.0,50.0,72.757389,70.0,13.400941,706547.0,8.63,1.17,1.984035,1.28,1.384604,2857.01,8440.0,0.0,891.184713,0.0,1557.482515,699580.0,113.0,0.0,11.959236,0.0,20.871592,9388.0,12.0
75%,NaN,23.0,5.0,499.0,NaN,2.591802e+10,36060000.0,0.0,535.0,66.0,1.0,601.0,2.591802e+10,85.0,21.0,19.0,45.0,107.0,61.0,0.051089,133.0,50.0,72.757389,70.0,13.400941,706547.0,8.63,1.17,1.984035,1.28,1.384604,2857.01,8440.0,0.0,891.184713,0.0,1557.482515,699580.0,113.0,0.0,11.959236,0.0,20.871592,9388.0,12.0
max,NaN,23.0,5.0,499.0,NaN,2.591802e+10,36060000.0,0.0,535.0,66.0,1.0,601.0,2.591802e+10,85.0,21.0,19.0,45.0,107.0,61.0,0.051089,133.0,50.0,72.757389,70.0,13.400941,706547.0,8.63,1.17,1.984035,1.28,1.384604,2857.01,8440.0,0.0,891.184713,0.0,1557.482515,699580.0,113.0,0.0,11.959236,0.0,20.871592,9388.0,12.0


In [ ]:
p12_z = med_12.drop(columns=['minutesToFallAsleep','dist_min','steps_min','pid'])

z_score_12 = stats.zscore(p12_z, nan_policy='omit')

abs_z_scores = np.abs(z_score_12)
filtered_entries = (abs_z_scores < 3).all(axis=1)
z_df12 = p12_z[filtered_entries]

#display(z_df12)

/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:2413: RuntimeWarning: Mean of empty slice
  mns = np.nanmean(a=a, axis=axis, keepdims=True)
/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:2419: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


In [ ]:
p13_z = med_13.drop(columns=['resthr','minutesToFallAsleep','dist_min','steps_min','pid'])
z_score_13 = stats.zscore(p13_z, nan_policy='omit')

In [ ]:
p14_z = med_14.drop(columns=['minutesToFallAsleep','dist_min','steps_min','pid'])
z_score_14 = stats.zscore(p14_z, nan_policy='omit')

In [ ]:
p15_z = med_15.drop(columns=['minutesToFallAsleep','dist_min','steps_min','pid'])
z_score_15 = stats.zscore(p15_z, nan_policy='omit')

In [ ]:
p16_z = med_16.drop(columns=['minutesToFallAsleep','dist_min','dist_median','steps_min','steps_median','pid'])
z_score_16 = stats.zscore(p16_z)

In [ ]:
from numpy import savetxt
"""
savetxt('p1zv2.csv', z_score_1, delimiter=',')
savetxt('p2zv2.csv', z_score_2, delimiter=',')
savetxt('p3zv2.csv', z_score_3, delimiter=',')
savetxt('p4zv2.csv', z_score_4, delimiter=',')
savetxt('p5zv2.csv', z_score_5, delimiter=',')
savetxt('p6zv2.csv', z_score_6, delimiter=',')
savetxt('p7zv2.csv', z_score_7, delimiter=',')
savetxt('p8zv2.csv', z_score_8, delimiter=',')
savetxt('p9zv2.csv', z_score_9, delimiter=',')
savetxt('p10zv2.csv', z_score_10, delimiter=',')
savetxt('p11zv2.csv', z_score_11, delimiter=',')
savetxt('p12zv2.csv', z_score_12, delimiter=',')
savetxt('p13zv2.csv', z_score_13, delimiter=',')
savetxt('p14zv2.csv', z_score_14, delimiter=',')
savetxt('p15zv2.csv', z_score_15, delimiter=',')
savetxt('p16zv2.csv', z_score_16, delimiter=',')
"""



"\nsavetxt('p1zv2.csv', z_score_1, delimiter=',')\nsavetxt('p2zv2.csv', z_score_2, delimiter=',')\nsavetxt('p3zv2.csv', z_score_3, delimiter=',')\nsavetxt('p4zv2.csv', z_score_4, delimiter=',')\nsavetxt('p5zv2.csv', z_score_5, delimiter=',')\nsavetxt('p6zv2.csv', z_score_6, delimiter=',')\nsavetxt('p7zv2.csv', z_score_7, delimiter=',')\nsavetxt('p8zv2.csv', z_score_8, delimiter=',')\nsavetxt('p9zv2.csv', z_score_9, delimiter=',')\nsavetxt('p10zv2.csv', z_score_10, delimiter=',')\nsavetxt('p11zv2.csv', z_score_11, delimiter=',')\nsavetxt('p12zv2.csv', z_score_12, delimiter=',')\nsavetxt('p13zv2.csv', z_score_13, delimiter=',')\nsavetxt('p14zv2.csv', z_score_14, delimiter=',')\nsavetxt('p15zv2.csv', z_score_15, delimiter=',')\nsavetxt('p16zv2.csv', z_score_16, delimiter=',')\n"

In [ ]:
zbin2 = np.digitize(z_score_2, [-0.333, 0.333])
print(zbin2)

for i in zbin2: 
  unique, counts = np.unique(zbin2[i], return_counts=True)
  print(np.asarray((unique, counts)).T)


[[2 0 0 0 0 0 2 2 2 2 ... 0 0 0 0 0 0 0 0 0 0]
 [2 0 0 1 1 0 1 1 1 1 ... 0 0 0 0 0 0 0 0 0 0]
 [2 0 0 2 1 0 0 0 0 0 ... 0 0 1 1 1 1 1 0 1 1]
 [0 1 2 2 0 0 0 0 0 0 ... 1 1 2 1 1 1 2 1 2 1]
 [0 2 1 1 0 0 1 1 1 1 ... 1 0 1 1 1 1 1 1 1 1]
 [0 2 2 0 0 0 2 2 2 2 ... 2 2 1 1 1 1 1 1 1 1]
 [1 2 2 0 0 0 1 1 1 1 ... 2 2 2 2 2 2 2 2 2 2]
 [1 0 2 0 0 0 2 2 0 2 ... 2 2 2 2 2 2 2 2 2 2]
 [0 0 2 1 0 0 2 2 1 2 ... 1 1 2 2 2 2 2 2 2 2]
 [0 0 2 1 0 0 1 1 1 1 ... 2 1 2 2 2 1 2 1 2 1]
 ...
 [1 2 2 1 1 2 1 1 1 1 ... 2 2 2 1 2 2 2 0 1 1]
 [0 0 0 2 2 2 2 1 2 2 ... 1 0 1 1 2 0 1 1 2 0]
 [0 0 2 1 2 2 2 2 2 2 ... 2 1 1 2 2 0 1 2 2 0]
 [0 0 1 1 2 2 2 2 2 2 ... 1 0 0 1 1 0 1 1 1 0]
 [0 0 0 2 2 2 2 2 2 2 ... 0 0 1 0 2 0 1 0 2 0]
 [0 0 0 2 1 2 1 1 1 1 ... 1 0 1 0 0 0 0 0 0 0]
 [0 0 0 2 1 2 1 1 2 1 ... 1 0 0 0 0 0 0 0 0 0]
 [2 2 0 0 2 2 2 1 2 2 ... 1 1 1 1 1 1 1 1 1 1]
 [0 2 1 2 2 2 0 0 0 0 ... 0 0 0 2 1 1 0 2 2 1]
 [0 0 2 2 2 2 0 0 0 0 ... 1 0 1 2 2 1 1 2 2 1]]
[[  0 837]
 [  1 236]
 [  2 371]]
[[  0 833]
 [  1 225

In [ ]:
zbin3 = np.digitize(z_score_3, [-0.333, 0.333])
unique, counts = np.unique(zbin3, return_counts=True)

print(np.asarray((unique, counts)).T)

[[   0 1224]
 [   1  908]
 [   2 1148]]


In [ ]:
zbin4 = np.digitize(z_score_4, [-0.333, 0.333])
unique, counts = np.unique(zbin4, return_counts=True)

print(np.asarray((unique, counts)).T)

[[  0 665]
 [  1 433]
 [  2 624]]


In [ ]:
zbin5 = np.digitize(z_score_5, [-0.333, 0.333])
unique, counts = np.unique(zbin5, return_counts=True)

print(np.asarray((unique, counts)).T)

[[   0 1211]
 [   1  767]
 [   2 1056]]


In [ ]:
zbin6 = np.digitize(z_score_6, [-0.333, 0.333])
unique, counts = np.unique(zbin6, return_counts=True)

print(np.asarray((unique, counts)).T)

[[   0 2559]
 [   1 1643]
 [   2 2399]]


In [ ]:
zbin7 = np.digitize(z_score_7, [-0.333, 0.333])
unique, counts = np.unique(zbin7, return_counts=True)

print(np.asarray((unique, counts)).T)

[[   0 2498]
 [   1 1671]
 [   2 2350]]


In [ ]:
zbin8 = np.digitize(z_score_8, [-0.333, 0.333])
unique, counts = np.unique(zbin8, return_counts=True)

print(np.asarray((unique, counts)).T)

[[   0 1973]
 [   1 1710]
 [   2 2139]]
